Here we are going to extract the basketball seasons from the [NBA Site](https://www.nba.com/stats/teams/traditional?Outcome=&SeasonType=Regular+Season&Season=2023-24).

1. To extract dataset from the site, we are going to use a web scraper, and chosen to go with Selenium. <br/>
   Read the docs to know more about [Selenium](https://selenium-python.readthedocs.io/installation.html)


In [ ]:
# import selenium.webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import Select
# import time

driver = webdriver.Chrome()
url = "https://www.nba.com/stats/teams/traditional?Outcome=&SeasonType=Regular+Season"

driver.get(url)

season_val = "2020-21"

season_drpdwn = Select(driver.find_element(By.CLASS_NAME,"DropDown_select__4pIg9"))
season_drpdwn.select_by_visible_text(season_val)

time.sleep(10)
# driver.quit()

In [ ]:
import xlsxwriter

workbook = xlsxwriter.Workbook("nba_stats.xlsx")
workbook = 